# Image Rectification
The goal of image rectificiation is to find the Homographies H and H' for the stereo images such that in the transformed image pai, the same world point appears on the same row.

### Import Statements

In [1]:
import cv2
import numpy as np
from image_rectification_helper import *

### Load Images

In [2]:
left_file_path = "/home/jo_wang/Desktop/ECE661/HW09/input_images/left.jpg"
right_file_path = "/home/jo_wang/Desktop/ECE661/HW09/input_images/right.jpg"
left_img = cv2.imread(left_file_path, 1)
right_img = cv2.imread(right_file_path, 1)
assert(left_img.shape == right_img.shape)

### Point Correspondences
Purpose: Establish Point 8 point correspondences between left and right image
* x_points (list): list of points in left image
* x_points_prime (list): list of points in right image
* Both lists look like this:
[[x1,y1], [x2,y2], [x3,y3], ..., [x8,y8]]

In [3]:
point_corr_image  = np.hstack((left_img, right_img))
width_adj = right_img.shape[1]
x_points = [[722, 347], [690, 466], [820, 338], [785, 453], [284,487], [280, 463], [477, 191], [622,185]]
x_prime_points = [[618,428], [599, 531], [750, 433], [717, 537], [265,457], [274,430], [505,182], [674,184]]
rainbow = [(211, 0, 148), (130, 0, 75), (255, 0, 0), (0, 255, 0), (0, 255, 255), (0, 127, 255), (0, 0, 255), (0, 0, 0)]

for i in range(len(x_points)):
    left_point = tuple(x_points[i])
    right_point = (x_prime_points[i][0] + width_adj, x_prime_points[i][1])
    cv2.circle(point_corr_image, left_point, 4, rainbow[i], -1)
    cv2.circle(point_corr_image, right_point, 4, rainbow[i], -1)
    cv2.line(point_corr_image, left_point, right_point, rainbow[i], 1)

# cv2.imwrite("point_correspondences.jpg", point_corr_image)

### Normalized 8-point Algorithm for F
Purpose: Given 8 image point correspondences (x, x'), determine the fundamental matrix F

Algorithm:
1. Normalization: Transform the image coordinates to: $\hat x_i = Tx_i$ and $\hat x'_i = T'x'_i$

    T = $\begin{bmatrix}
            s & 0 & -s\bar x\\
            0 & s & -s\bar y\\
            0 & 0 & 1\\
        \end{bmatrix}$
    $\bar x$ and $ \bar y$ : mean of x and y coordinates respectively
    $D$ : list of distances from each point to $(\bar x,\bar y)$
    $\bar D$: mean of D
    $s = \frac{\sqrt 2}{\bar D}\\$

2. Find $\hat F$ from $A \hat F = 0$

   $A \in \mathbb{R}^{8 \times 9}$, $\hat F \in \mathbb{R}^{9}$

3. Constraint Enforcement: Ensure that the rank($\hat F$) = 2

   $\hat F = UDV^T$, $D = diag(r,s,t)$
   $ \hat F' = U diag(r,s,0) V^T$

4. Denormalization: $F = T'^T \hat F' T$

In [4]:
f = compute_F(x_points, x_prime_points)
print(f)

[[ 1.39338093e-10  7.56539894e-11 -7.05215025e-05]
 [ 1.87247366e-10 -3.71411469e-11 -1.33666126e-04]
 [ 4.31441950e-05  1.05017163e-04  1.02776213e+00]]
